In [ ]:
# px는 마우스를 갖다 대면 수치가 보이고, 확대/축소가 가능한 인터랙티브 그래프
# pykrx import stock 한국 거래소(KRX)의 주식 데이터를 가져오는 마법 같은 도구
# datetime import datetime: 주가 데이터를 가져올 때 "오늘부터 한 달 전까지"처럼 날짜를 계산하기 위해 필요
#'{:.2f}'.format 소수점 둘쨰자리까지만
import pandas as pd
#import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from pykrx import stock
from datetime import datetime

# 출력 옵션 설정
pd.options.display.float_format = '{:.2f}'.format

print("✅ 라이브러리 로딩 완료!")

✅ 라이브러리 로딩 완료!


In [ ]:
# 파이썬의 딕셔너리 형식, 자산군, 비중, ETF, 티커가 열값이고, 각 열값 밑으로 VALUE값
# 지정한 변수의 INFO값
portfolio_df = pd.DataFrame({
    '자산군': ['주식', '장기채권', '중기채권', '원자재', '금'],
    '비중(%)': [30, 40, 15, 7.5, 7.5],
    'ETF명': [
        'KODEX 200',
        'ACE 미국30년국채액티브', 
        
        'TIGER 미국채10년선물',
        'KODEX 골드선물(H)',
        'KODEX 골드선물(H)'
    ],
    '티커': ['133690', '411060', '356540', '130680', '261240']
})
portfolio_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   자산군     5 non-null      object 
 1   비중(%)   5 non-null      float64
 2   ETF명    5 non-null      object 
 3   티커      5 non-null      object 
dtypes: float64(1), object(3)
memory usage: 288.0+ bytes


In [ ]:
# 파이차트를 그리되, portfolio_df 변수에 저장된 데이터를 쓴다. 
# values는 파이차트를 그릴 때 비중%기준으로 파이조각을 나누며, 그안에 들어가는 이름 태그는 자산군 열에 있는 값으로 보여준다. 상단 차트의 타이틀명칭은 올웨어 포트폴리오 자산 배분
# hole에 들어간 0.3은 구멍 뚫린 정도. 0이면 가운데 막ㅎ힌 일반 파이차트이다.
# 딱딱 끊어지는 항목들 discrete. 그 항목들의 컬러들은 Qualitative범주형 컬러 모음인데 set3를 쓰겠다.
# fig.update_traces는 이미 만들어진 모양과 표시방식을 수정하겠다. inside 안쪽에다 넣겠다. 안쪽에다 퍼센트와 라벨값을 넣겠다.
fig = px.pie(
    portfolio_df,
    values='비중(%)',
    names='자산군',
    title='올웨더 포트폴리오 자산 배분',
    hole=0.3,
    color_discrete_sequence=px.colors.qualitative.Set3
)

fig.update_traces(
    textposition='inside',
    textinfo='percent+label'
)

fig.show()

In [ ]:
# 시작날짜, 종료날짜를 변수로 저장하고 f-string을 사용해 변수값을 문자열안에 넣고 출력
# 시간이 걸리는 작업이므로 상단에 무슨 작업을 진행하는지 알려주기 위함
# price_data라는 빈 딕셔너리를 만든다. 
# portfolio_df.iterrows()1줄씩 읽는 번역기. idx몇번째 줄, row 그 줄에 들어있는 정보
# row['티커']는 현재 읽고 있는 줄에서 티커의 번호만 뽑아내라.asset_name- 자산군 이름을 꺼내라.
# ticker = row['티커'] 현재 읽고 있는 줄에서 티커 값을 골라내고 ticker 값에 저장해라
# asset_name = row['자산군'] 현재 읽고 있는 줄에서 자산군 컬럼 값을 골라내어 해당 변수에 저장
# print(f" └ {asset_name} ({ticker}) 수집 중...", end=" ")는 문자열 안에 변수를 넣어서 출력하되, end"" 줄바꿈 하지 말고
start_date = '20230101'
end_date = '20260131'

print(f"📥 {start_date} ~ {end_date} 기간의 데이터를 가져옵니다...\n")

price_data = {}

for idx, row in portfolio_df.iterrows():
    ticker = row['티커']
    asset_name = row['자산군']
    
    print(f"  └ {asset_name} ({ticker}) 수집 중...", end=" ")
    
    try:
        df = stock.get_etf_ohlcv_by_date(start_date, end_date, ticker)
        price_data[asset_name] = df['종가']
        print(f"✅ 완료 ({len(df)}개)")
    except Exception as e:
        print(f"❌ 오류: {e}")

price_df = pd.DataFrame(price_data)

print(f"\n✅ 전체 데이터 수집 완료! (총 {len(price_df)}일)")
print("\n📊 최근 5일 종가:")
print(price_df.tail())

📥 20230101 ~ 20260131 기간의 데이터를 가져옵니다...

  └ 주식 (133690) 수집 중... ✅ 완료 (752개)
  └ 장기채권 (411060) 수집 중... ✅ 완료 (752개)
  └ 중기채권 (356540) 수집 중... ✅ 완료 (752개)
  └ 원자재 (130680) 수집 중... ✅ 완료 (752개)
  └ 금 (261240) 수집 중... ✅ 완료 (752개)

✅ 전체 데이터 수집 완료! (총 752일)

📊 최근 5일 종가:
                주식   장기채권    중기채권   원자재      금
날짜                                            
2026-01-26  163530  33380  104390  4080  14620
2026-01-27  165495  34205  104440  4020  14655
2026-01-28  165035  35315  104570  4165  14440
2026-01-29  164915  37890  104315  4240  14465
2026-01-30  163585  35440  104130  4250  14610
